In [3]:
import numpy as np
import os
import time
import math
import random
import heapq
from collections import defaultdict
from typing import Dict, List, Tuple, Set, Any
from tqdm import tqdm

# Global constants
R_EARTH = 6371  # Earth radius (km)
LIGHT_SPEED = 299792.458  # Speed of light (km/s)
ATMOSPHERE_HEIGHT = 80  # Atmosphere height (km)
R_ATMOSPHERE = R_EARTH + ATMOSPHERE_HEIGHT
MIN_ELEVATION = 10  # Minimum elevation angle (degrees) for ground station visibility

# Set MPC ground station location (longitude, latitude in radians)
MPC_LON = math.radians(42)  # Beijing longitude
MPC_LAT = math.radians(-73)   # Beijing latitude
MPC_LOCATION = (MPC_LAT, MPC_LON)

# Data paths
DATA_DIR = "data"
# Output directory
OUTPUT_DIR = "data"
# Number of simulations
NUM_SIMULATIONS = 2000
# Timestamp
TIMESTAMP = 0

def get_original_satellite_id(virtual_id: int, num_satellites: int) -> int:
    """Get original satellite ID from virtual node ID"""
    return virtual_id % num_satellites

def geodetic_to_cartesian(lat: float, lon: float, height: float) -> Tuple[float, float, float]:
    """Convert geographic coordinates to Cartesian coordinates"""
    r = R_EARTH + height
    x = r * math.cos(lat) * math.cos(lon)
    y = r * math.cos(lat) * math.sin(lon)
    z = r * math.sin(lat)
    return x, y, z

def calculate_distance(pos1: Tuple[float, float, float], 
                      pos2: Tuple[float, float, float]) -> float:
    """Calculate Euclidean distance between two points"""
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(pos1, pos2)))

def calculate_satellite_distance(sat1_id: int, sat2_id: int,
                               satellite_locations: Dict,
                               current_timestamp: int,
                               satellite_params: Dict) -> float:
    """Calculate distance between two satellites"""
    # Get satellite positions and heights
    lon1, lat1 = satellite_locations[current_timestamp][sat1_id]
    lon2, lat2 = satellite_locations[current_timestamp][sat2_id]
    height1 = satellite_params[sat1_id]['height']
    height2 = satellite_params[sat2_id]['height']
    
    # Convert to Cartesian coordinates
    pos1 = geodetic_to_cartesian(lat1, lon1, height1)
    pos2 = geodetic_to_cartesian(lat2, lon2, height2)
    
    # Calculate Euclidean distance
    return calculate_distance(pos1, pos2)

def check_satellite_visibility(sat1_id: int, sat2_id: int,
                             satellite_locations: Dict,
                             current_timestamp: int,
                             satellite_params: Dict) -> bool:
    """Check if two satellites are visible to each other"""
    # Get satellite heights
    height1 = satellite_params[sat1_id]['height']
    height2 = satellite_params[sat2_id]['height']
    
    # Calculate maximum visible distance
    r1 = R_EARTH + height1
    r2 = R_EARTH + height2
    max_distance = math.sqrt(r1 ** 2 - R_ATMOSPHERE ** 2) + math.sqrt(r2 ** 2 - R_ATMOSPHERE ** 2)
    
    # Calculate actual distance
    distance = calculate_satellite_distance(
        sat1_id, sat2_id, satellite_locations, current_timestamp, satellite_params
    )
    
    return distance <= max_distance

def calculate_elevation_angle(ground_pos: Tuple[float, float, float], 
                             sat_pos: Tuple[float, float, float]) -> float:
    """Calculate elevation angle (degrees) from ground station to satellite"""
    # Calculate vector from ground to satellite
    vector = tuple(sat - ground for sat, ground in zip(sat_pos, ground_pos))
    
    # Calculate vector length
    vector_length = math.sqrt(sum(v**2 for v in vector))
    
    # Calculate ground station normal vector (assuming Earth is spherical)
    # Normal vector is the unit vector of ground station position
    normal = tuple(coord / R_EARTH for coord in ground_pos)
    normal_length = math.sqrt(sum(n**2 for n in normal))
    
    # Calculate angle between normal vector and connection vector
    dot_product = sum(n * v for n, v in zip(normal, vector))
    cos_angle = dot_product / (normal_length * vector_length)
    
    # Convert cosine to angle in degrees, subtract from 90 to get elevation
    angle_rad = math.acos(min(1.0, max(-1.0, cos_angle)))
    elevation_angle = 90 - math.degrees(angle_rad)
    
    return elevation_angle

def is_satellite_visible_from_ground(ground_pos: Tuple[float, float, float], 
                                    sat_pos: Tuple[float, float, float]) -> bool:
    """Check if satellite is visible from ground station (based on minimum elevation)"""
    elevation = calculate_elevation_angle(ground_pos, sat_pos)
    return elevation >= MIN_ELEVATION

def build_network_graph(inter_topology: List, intra_topology: Dict,
                       satellite_locations: Dict, satellite_params: Dict,
                       current_timestamp: int, num_satellites: int) -> Dict[Any, Dict[Any, float]]:
    """Build satellite network graph"""
    graph = defaultdict(dict)
    
    # Add intra-domain links
    for grid_id, connections in intra_topology.items():
        for src, dst in connections:
            src_sat = get_original_satellite_id(src, num_satellites)
            dst_sat = get_original_satellite_id(dst, num_satellites)
            
            # Calculate distance between satellites
            distance = calculate_satellite_distance(
                src_sat, dst_sat, satellite_locations, current_timestamp, satellite_params
            )
            
            # Add bidirectional edges
            graph[src_sat][dst_sat] = distance
            graph[dst_sat][src_sat] = distance
    
    # Add inter-domain links
    for (node1, node2), _ in inter_topology:
        sat1 = get_original_satellite_id(node1, num_satellites)
        sat2 = get_original_satellite_id(node2, num_satellites)
        
        # Calculate distance between satellites
        distance = calculate_satellite_distance(
            sat1, sat2, satellite_locations, current_timestamp, satellite_params
        )
        
        # Add bidirectional edges
        graph[sat1][sat2] = distance
        graph[sat2][sat1] = distance
    
    return graph

def add_ground_station_to_graph(graph: Dict[Any, Dict[Any, float]], 
                              ground_location: Tuple[float, float],
                              satellite_locations: Dict, satellite_params: Dict,
                              current_timestamp: int) -> Dict[Any, Dict[Any, float]]:
    """Add ground station to the network graph"""
    ground_lat, ground_lon = ground_location
    ground_pos = geodetic_to_cartesian(ground_lat, ground_lon, 0)
    
    # Add edges for all visible satellites
    for sat_id in satellite_locations[current_timestamp]:
        # Get satellite position
        sat_lon, sat_lat = satellite_locations[current_timestamp][sat_id]
        sat_height = satellite_params[sat_id]['height']
        sat_pos = geodetic_to_cartesian(sat_lat, sat_lon, sat_height)
        
        # Check visibility
        if is_satellite_visible_from_ground(ground_pos, sat_pos):
            # Calculate distance
            distance = calculate_distance(ground_pos, sat_pos)
            
            # Add edge
            graph[sat_id]['ground_station'] = distance
            graph['ground_station'][sat_id] = distance
    
    return graph

def find_shortest_path(graph: Dict[Any, Dict[Any, float]], 
                      start: Any, end: Any) -> Tuple[List[Any], float]:
    """Find shortest path from start to end using Dijkstra's algorithm"""
    if start not in graph or end not in graph:
        return [], float('inf')
    
    # Initialize
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    previous = {node: None for node in graph}
    priority_queue = [(0, start)]
    visited = set()
    
    # Dijkstra's algorithm
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        
        if current_node == end:
            break
            
        if current_node in visited:
            continue
            
        visited.add(current_node)
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))
    
    # Reconstruct path
    path = []
    current = end
    
    while current:
        path.append(current)
        current = previous[current]
    
    path.reverse()
    
    # If no path was found
    if not path or path[0] != start:
        return [], float('inf')
    
    return path, distances[end]

def calculate_network_transmission_time(source_sat: int, 
                                      satellite_locations: Dict, satellite_params: Dict,
                                      inter_topology: List, intra_topology: Dict,
                                      current_timestamp: int, num_satellites: int) -> float:
    """Calculate transmission time from satellite to ground station through the network"""
    # Build network graph
    graph = build_network_graph(inter_topology, intra_topology, 
                              satellite_locations, satellite_params, 
                              current_timestamp, num_satellites)
    
    # Add ground station to graph
    graph = add_ground_station_to_graph(graph, MPC_LOCATION, 
                                      satellite_locations, satellite_params, 
                                      current_timestamp)
    
    # Find shortest path using Dijkstra's algorithm
    path, total_distance = find_shortest_path(graph, source_sat, 'ground_station')
    
    # If no path is found
    if not path:
        # Fall back to direct line-of-sight distance calculation
        lon, lat = satellite_locations[current_timestamp][source_sat]
        height = satellite_params[source_sat]['height']
        sat_pos = geodetic_to_cartesian(lat, lon, height)
        ground_pos = geodetic_to_cartesian(MPC_LAT, MPC_LON, 0)
        total_distance = calculate_distance(sat_pos, ground_pos)
    
    # Calculate transmission time
    return total_distance / LIGHT_SPEED

def load_data(input_dir: str, timestamp: int):
    """Load topology and satellite data for specified timestamp"""
    # Load inter-domain topology
    all_inter_topology = np.load(
        os.path.join(input_dir, 'all_inter_topology_573_11_11_distancedt_global_max.npy'), 
        allow_pickle=True
    ).item()
    
    # Load intra-domain topology
    all_intra_topology = np.load(
        os.path.join(input_dir, 'all_intra_topology_573_11_11_distancedt_global_max.npy'), 
        allow_pickle=True
    ).item()
    
    # Load satellite parameters
    supply_data = np.load(
        os.path.join(input_dir, "eval1_573_jinyao_24k_half.npy"), 
        allow_pickle=True
    )
    
    # Extract satellite parameters
    num_satellites = len(supply_data)
    satellite_params = {}
    satellite_locations = {timestamp: {}}
    
    for idx, data in enumerate(supply_data):
        param, random_numbers, _, sat_location, _ = data
        
        satellite_params[idx] = {
            'height': param[0],
            'inclination': param[1],
            'alpha0': param[2],
            'initial_slot': random_numbers
        }
        
        # Only load position for current timestamp
        satellite_locations[timestamp][idx] = sat_location[timestamp]
    
    # Load grid satellite coverage data
    grid_satellites = np.load(
        os.path.join(input_dir, "new_grid_satellites.npy"), 
        allow_pickle=True
    ).item()
    
    return all_inter_topology[timestamp], all_intra_topology[timestamp], satellite_params, satellite_locations, grid_satellites[timestamp], num_satellites

def find_domain_for_satellites(sat1: int, sat2: int, inter_topology: List, intra_topology: Dict, num_satellites: int) -> Tuple[str, Tuple]:
    """Determine if two satellites have inter-domain or intra-domain connection"""
    # First check inter-domain topology
    for (node1, node2), (grid1, grid2) in inter_topology:
        s1 = get_original_satellite_id(node1, num_satellites)
        s2 = get_original_satellite_id(node2, num_satellites)
        if (s1 == sat1 and s2 == sat2) or (s1 == sat2 and s2 == sat1):
            return "inter", (grid1, grid2)
    
    # Then check intra-domain topology
    for grid_id, connections in intra_topology.items():
        for src, dst in connections:
            s1 = get_original_satellite_id(src, num_satellites)
            s2 = get_original_satellite_id(dst, num_satellites)
            if (s1 == sat1 and s2 == sat2) or (s1 == sat2 and s2 == sat1):
                return "intra", (grid_id,)
    
    # No connection found
    return "unknown", ()

def get_intra_domain_neighbors(satellite_id: int, grid_id: int, intra_topology: Dict, num_satellites: int) -> List[int]:
    """Get neighbor satellites of a satellite within its domain"""
    neighbors = []
    
    if grid_id in intra_topology:
        for src, dst in intra_topology[grid_id]:
            src_sat = get_original_satellite_id(src, num_satellites)
            dst_sat = get_original_satellite_id(dst, num_satellites)
            
            if src_sat == satellite_id:
                neighbors.append(dst_sat)
            elif dst_sat == satellite_id:
                neighbors.append(src_sat)
    
    return neighbors

def find_inter_domain_connections(satellite_id: int, inter_topology: List, num_satellites: int) -> List[Tuple[int, int]]:
    """Find all inter-domain connections involving the satellite"""
    connections = []
    
    for (node1, node2), (grid1, grid2) in inter_topology:
        sat1 = get_original_satellite_id(node1, num_satellites)
        sat2 = get_original_satellite_id(node2, num_satellites)
        
        if sat1 == satellite_id:
            connections.append((sat2, grid2))
        elif sat2 == satellite_id:
            connections.append((sat1, grid1))
    
    return connections

def handle_inter_domain_failure(failed_sat1: int, failed_sat2: int, 
                              grids: Tuple[int, int],
                              satellite_params: Dict,
                              satellite_locations: Dict,
                              grid_satellites: Dict,
                              intra_topology: Dict,
                              current_timestamp: int,
                              num_satellites: int) -> Tuple[List[Tuple], float]:
    """Handle inter-domain link failure"""
    # Start timing
    start_time = time.time()
    
    grid1, grid2 = grids
    
    # Determine roles of failed satellites (which one is in grid1, which one is in grid2)
    if failed_sat1 in [get_original_satellite_id(s, num_satellites) for s in grid_satellites[grid1]]:
        src_sat = failed_sat1
        dst_sat = failed_sat2
        src_grid = grid1
        dst_grid = grid2
    else:
        src_sat = failed_sat2
        dst_sat = failed_sat1
        src_grid = grid2
        dst_grid = grid1
    
    # Get intra-domain neighbors of dst_sat in target grid
    dst_neighbors = get_intra_domain_neighbors(dst_sat, dst_grid, intra_topology, num_satellites)
    
    # Find a replacement satellite in the target grid
    best_candidate = None
    best_score = float('inf')  # Lower is better
    
    for candidate_id in grid_satellites[dst_grid]:
        candidate_real_id = get_original_satellite_id(candidate_id, num_satellites)
        
        # Skip the failed satellites themselves
        if candidate_real_id == dst_sat or candidate_real_id == src_sat:
            continue
            
        # Check if candidate can establish connection with source satellite
        if not check_satellite_visibility(
            src_sat, candidate_real_id, satellite_locations, current_timestamp, satellite_params
        ):
            continue
        
        # Check if candidate can establish connections with all intra-domain neighbors of target satellite
        can_connect_to_all_neighbors = True
        for neighbor in dst_neighbors:
            if not check_satellite_visibility(
                candidate_real_id, neighbor, satellite_locations, current_timestamp, satellite_params
            ):
                can_connect_to_all_neighbors = False
                break
        
        if not can_connect_to_all_neighbors:
            continue
        
        # Calculate distance as score (closer is better)
        distance_to_src = calculate_satellite_distance(
            src_sat, candidate_real_id, satellite_locations, current_timestamp, satellite_params
        )
        
        if distance_to_src < best_score:
            best_score = distance_to_src
            best_candidate = candidate_real_id
    
    # If a suitable replacement satellite is found
    if best_candidate is not None:
        # New connection relationships
        changes = [
            ('add', src_sat, best_candidate),  # Add connection from source to new satellite
        ]
        
        # Add connections from new satellite to neighbors of original satellite
        for neighbor in dst_neighbors:
            changes.append(('add', best_candidate, neighbor))
            changes.append(('remove', dst_sat, neighbor))  # Remove connections from failed satellite to neighbors
        
        # End timing and return results
        processing_time = time.time() - start_time
        return changes, processing_time
    else:
        # No suitable replacement found
        processing_time = time.time() - start_time
        return [], processing_time

def handle_intra_domain_failure(failed_sat1: int, failed_sat2: int, 
                              grid_id: int,
                              satellite_params: Dict,
                              satellite_locations: Dict,
                              grid_satellites: Dict,
                              intra_topology: Dict,
                              inter_topology: List,
                              current_timestamp: int,
                              num_satellites: int) -> Tuple[List[Tuple], float]:
    """Handle intra-domain link failure"""
    # Start timing
    start_time = time.time()
    
    # Determine intra-domain neighbors of both failed satellites
    neighbors_sat1 = [n for n in get_intra_domain_neighbors(failed_sat1, grid_id, intra_topology, num_satellites) if n != failed_sat2]
    neighbors_sat2 = [n for n in get_intra_domain_neighbors(failed_sat2, grid_id, intra_topology, num_satellites) if n != failed_sat1]
    
    # Determine inter-domain connections of failed satellites
    inter_connections_sat1 = find_inter_domain_connections(failed_sat1, inter_topology, num_satellites)
    inter_connections_sat2 = find_inter_domain_connections(failed_sat2, inter_topology, num_satellites)
    
    # Find a replacement satellite in the same grid
    best_candidate = None
    best_score = float('inf')  # Lower is better
    
    for candidate_id in grid_satellites[grid_id]:
        candidate_real_id = get_original_satellite_id(candidate_id, num_satellites)
        
        # Skip the failed satellites themselves
        if candidate_real_id == failed_sat1 or candidate_real_id == failed_sat2:
            continue
        
        # Check if candidate is already in intra-domain topology (avoid using satellites with other connections)
        if candidate_real_id in get_intra_domain_neighbors(failed_sat1, grid_id, intra_topology, num_satellites) or \
           candidate_real_id in get_intra_domain_neighbors(failed_sat2, grid_id, intra_topology, num_satellites):
            continue
            
        # Check if candidate can establish connections with neighbors of both failed satellites
        can_connect = True
        
        for neighbor in neighbors_sat1 + neighbors_sat2:
            if not check_satellite_visibility(
                candidate_real_id, neighbor, satellite_locations, current_timestamp, satellite_params
            ):
                can_connect = False
                break
        
        # Check if candidate can establish connections with inter-domain connections of failed satellites
        for connected_sat, _ in inter_connections_sat1 + inter_connections_sat2:
            if not check_satellite_visibility(
                candidate_real_id, connected_sat, satellite_locations, current_timestamp, satellite_params
            ):
                can_connect = False
                break
        
        if not can_connect:
            continue
        
        # Calculate total distance to all relevant satellites as score
        total_distance = 0
        for neighbor in neighbors_sat1 + neighbors_sat2:
            total_distance += calculate_satellite_distance(
                candidate_real_id, neighbor, satellite_locations, current_timestamp, satellite_params
            )
        
        for connected_sat, _ in inter_connections_sat1 + inter_connections_sat2:
            total_distance += calculate_satellite_distance(
                candidate_real_id, connected_sat, satellite_locations, current_timestamp, satellite_params
            )
        
        if total_distance < best_score:
            best_score = total_distance
            best_candidate = candidate_real_id
    
    # If a suitable replacement satellite is found
    if best_candidate is not None:
        # New connection relationships
        changes = []
        
        # Add connections from new satellite to neighbors of failed satellites
        for neighbor in neighbors_sat1:
            changes.append(('add', best_candidate, neighbor))
            changes.append(('remove', failed_sat1, neighbor))
        
        for neighbor in neighbors_sat2:
            changes.append(('add', best_candidate, neighbor))
            changes.append(('remove', failed_sat2, neighbor))
        
        # Add connections from new satellite to inter-domain connections of failed satellites
        for connected_sat, connected_grid in inter_connections_sat1:
            changes.append(('add', best_candidate, connected_sat))
            changes.append(('remove', failed_sat1, connected_sat))
        
        for connected_sat, connected_grid in inter_connections_sat2:
            changes.append(('add', best_candidate, connected_sat))
            changes.append(('remove', failed_sat2, connected_sat))
        
        # End timing and return results
        processing_time = time.time() - start_time
        return changes, processing_time
    else:
        # No suitable replacement found
        processing_time = time.time() - start_time
        return [], processing_time

def collect_all_links(inter_topology, intra_topology, num_satellites):
    """Collect all possible failure links"""
    all_links = []
    
    # Collect inter-domain links
    for (node1, node2), (grid1, grid2) in inter_topology:
        sat1 = get_original_satellite_id(node1, num_satellites)
        sat2 = get_original_satellite_id(node2, num_satellites)
        all_links.append((sat1, sat2, "inter", (grid1, grid2)))
    
    # Collect intra-domain links
    for grid_id, connections in intra_topology.items():
        for src, dst in connections:
            sat1 = get_original_satellite_id(src, num_satellites)
            sat2 = get_original_satellite_id(dst, num_satellites)
            all_links.append((sat1, sat2, "intra", (grid_id,)))
    
    return all_links

def simulate_link_failures(num_simulations=2000):
    """Simulate specified number of link failures and calculate delays"""
    # Create output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Load data
    print("Loading satellite data...")
    inter_topology, intra_topology, satellite_params, satellite_locations, grid_satellites, num_satellites = load_data(DATA_DIR, TIMESTAMP)
    
    # Collect all possible links
    print("Collecting network links...")
    all_links = collect_all_links(inter_topology, intra_topology, num_satellites)
    
    if not all_links:
        print("Error: No links found!")
        return
    
    print(f"Found {len(all_links)} links")
    print(f"Starting simulation of {num_simulations} random link failures...")
    
    # Results storage
    individual_results = []
    
    # Perform simulations
    for i in tqdm(range(num_simulations)):
        # Randomly select a link as failed link
        failed_link = random.choice(all_links)
        failed_sat1, failed_sat2, link_type, domain_info = failed_link
        
        # 1. Calculate failure notification time (time for failure info to reach MPC)
        # Use network path calculation instead of direct distance
        notify_delay = calculate_network_transmission_time(
            failed_sat1, satellite_locations, satellite_params, 
            inter_topology, intra_topology, TIMESTAMP, num_satellites
        ) * 1000  # Convert to milliseconds
        
        # 2. MPC computation time
        if link_type == "inter":
            changes, sdn_delay = handle_inter_domain_failure(
                failed_sat1, failed_sat2, domain_info,
                satellite_params, satellite_locations, grid_satellites,
                intra_topology, TIMESTAMP, num_satellites
            )
        else:  # intra
            changes, sdn_delay = handle_intra_domain_failure(
                failed_sat1, failed_sat2, domain_info[0],
                satellite_params, satellite_locations, grid_satellites,
                intra_topology, inter_topology, TIMESTAMP, num_satellites
            )
        
        sdn_delay *= 1000  # Convert to milliseconds
        
        # 3. Deployment delay (time for configuration info to be sent back to satellites)
        deploy_delay = 0
        if changes:
            # Find all satellites that need configuration
            satellites_to_configure = set()
            for action, sat1, sat2 in changes:
                satellites_to_configure.add(sat1)
                satellites_to_configure.add(sat2)
            
            # Calculate transmission time to each satellite (using network path)
            max_deploy_delay = 0
            for sat_id in satellites_to_configure:
                # Calculate network transmission time from MPC to satellite
                # Note: We're using the same network but considering the reverse path
                path_delay = calculate_network_transmission_time(
                    sat_id, satellite_locations, satellite_params,
                    inter_topology, intra_topology, TIMESTAMP, num_satellites
                ) * 1000  # Convert to milliseconds
                
                max_deploy_delay = max(max_deploy_delay, path_delay)
            
            deploy_delay = max_deploy_delay
        else:
            # If no changes, use conservative estimate
            deploy_delay = notify_delay
        
        # 4. Total delay
        total_delay = notify_delay + sdn_delay + deploy_delay
        
        # Save this simulation result
        result = {
            "link_type": "inter-domain" if link_type == "inter" else "intra-domain",
            "failed_satellites": (failed_sat1, failed_sat2),
            "domain_info": domain_info,
            "notify_delay": notify_delay,
            "sdn_delay": sdn_delay,
            "deploy_delay": deploy_delay,
            "total_delay": total_delay,
            "num_changes": len(changes),
            "success": len(changes) > 0
        }
        
        individual_results.append(result)
    
    # Calculate summary statistics
    success_count = sum(1 for r in individual_results if r["success"])
    success_rate = success_count / len(individual_results) if individual_results else 0
    
    avg_notify_delay = sum(r["notify_delay"] for r in individual_results) / len(individual_results) if individual_results else 0
    avg_sdn_delay = sum(r["sdn_delay"] for r in individual_results) / len(individual_results) if individual_results else 0
    avg_deploy_delay = sum(r["deploy_delay"] for r in individual_results) / len(individual_results) if individual_results else 0
    avg_total_delay = sum(r["total_delay"] for r in individual_results) / len(individual_results) if individual_results else 0
    
    # Summary results
    summary = {
        "total_simulations": len(individual_results),
        "success_count": success_count,
        "success_rate": success_rate,
        "avg_notify_delay": avg_notify_delay,
        "avg_sdn_delay": avg_sdn_delay,
        "avg_deploy_delay": avg_deploy_delay,
        "avg_total_delay": avg_total_delay
    }
    
    # Organize final results
    results = {
        "summary": summary,
        "individual_results": individual_results
    }
    
    # Save results
    np.save(os.path.join(OUTPUT_DIR, "link_failure_recovery_results.npy"), results)
    
    # Print summary statistics
    print("\n============= Simulation Results Summary =============")
    print(f"Total simulations: {len(individual_results)}")
    print(f"Successful recoveries: {success_count} ({success_rate*100:.2f}%)")
    print("\nAverage delay statistics (milliseconds):")
    print(f"Failure notification delay: {avg_notify_delay:.2f} ms")
    print(f"MPC computation delay: {avg_sdn_delay:.2f} ms")
    print(f"Deployment instruction delay: {avg_deploy_delay:.2f} ms")
    print(f"Total recovery delay: {avg_total_delay:.2f} ms")
    
    print(f"\nResults saved to {os.path.join(OUTPUT_DIR, 'link_failure_recovery_results.npy')}")
    
    return results

if __name__ == "__main__":
    # Set random seed to ensure reproducible results (optional)
    # random.seed(42)
    
    start_time = time.time()
    simulate_link_failures(NUM_SIMULATIONS)
    print(f"\nTotal runtime: {time.time() - start_time:.2f} seconds")

加载卫星数据...
收集网络链路...
共找到 1571 条链路
开始模拟 2000 次随机链路故障...


100%|██████████| 2000/2000 [02:49<00:00, 11.78it/s]


============= 模拟结果统计 =============
总模拟次数: 2000
成功修复次数: 2000 (100.00%)

平均延迟统计 (毫秒):
故障通知延迟: 61.65 ms
MPC计算延迟: 0.55 ms
部署指令延迟: 66.00 ms
总恢复延迟: 128.19 ms

结果已保存至 data/link_failure_recovery_results.npy

总运行时间: 170.71秒
